 # PVRP

従来のVRPでは、通常、計画期間は1日です。 Period Vehicle Routing Problem（PVRP）の場合、従来のVRPは計画期間をM日まで延長することにより一般化されます。

目的：目的は、車両群とすべての顧客に供給するために必要な移動時間の合計を最小限にすることです。

### データの読み方
index =顧客番号

x = x座標

y = y座標

d =サービス期間

q =需要

f =訪問の頻度

a =可能な訪問の組み合わせの数

list =すべての可能な訪問の組み合わせのリスト

 ## 定式化

$$
minimize \sum_{ij}^{}c_{ij}x_{ijt} \quad　(i<j)
$$

$$
s. t \sum_{j\in{N}}^{}x_{0jt} \quad <= 2m        ({t} \in {T})
$$

$$
 \sum_{i}^{}x_{ijt} \quad+\sum_{j}^{}x_{jit} \quad = 2a_{ipt}y_{ip}
$$

$$
\sum_{i}^{}y_{ip}\quad = 1
$$

$$
\sum_{{i,j}\in{S}}^{}x_{ij}<=|S|-N(S)       (S\in{(2,3...n)}:S>=3)
$$

 ### ●目的
 移動時間の合計（コスト）を最小化


 ### ●制約
点０からでる枝の数は２m本

必ず１度は全ての点を通る

必ずパターンのどれか一つを選ぶ

部分閉路禁止

 ### ●変数
**$C_{ij}$:点iから点jに移動するときの費用**

**$x_{ijt}$:運搬車が点iから点jにt日に移動する回数を表す変数**

**$y_{ip}$:どのパターン（p）で点iに訪れるか**

＊mは車両数

In [19]:
with open('c-pvrp/p01', 'r') as f:
    lines = f.read().splitlines()

low_0 = lines[0].split(" ")
vehicles= int(low_0[1])
customers = int(low_0[2]) 
days = int(low_0[3])
#max_load_ve

print(low_0)
m = low_0[1]
n = low_0[2]
d = low_0[3]

['1', '3', '51', '2']


In [20]:
index , x , y , d , q , f , a,li= [] , [] , [] , [] , [] , [] , [] , []

#index	=	customer number
#x	=	x coordinate
#y	=	y coordinate
#d	=	service duration
#q	=	demand
#f	=	frequency of visit
#a	=	number of possible visit combinations
#li	=	list of all possible visit combinations

for i in range(customers+1):
    low = lines[days+1+i].split(" ")
    #print(low)
    low = [j for j in low if j != '']
    #print(low)
    index.append(low[0])
    x.append(int(low[1]))
    y.append(int(low[2]))
    d.append(int(low[3]))
    q.append(int(low[4]))
    f.append(int(low[5])) 
    a.append(int(low[6]))
    li.append(low[7:])

## 必要なライブラリのインポート 

In [21]:
from gurobipy import *
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import networkx

In [22]:
f = open("c-pvrp/p01").read()
print()
L = [tuple(map(int , i.split())) for i in f.split("\n")]
print(L)

vehicles= int(L[0][1])
customers = int(L[0][2]) 
days = int(L[0][3])


[(1, 3, 51, 2), (0, 160), (0, 160), (0, 30, 40, 0, 0, 0, 0), (1, 37, 52, 0, 7, 1, 2, 1, 2), (2, 49, 49, 0, 30, 1, 2, 1, 2), (3, 52, 64, 0, 16, 1, 2, 1, 2), (4, 20, 26, 0, 9, 1, 2, 1, 2), (5, 40, 30, 0, 21, 1, 2, 1, 2), (6, 21, 47, 0, 15, 1, 2, 1, 2), (7, 17, 63, 0, 19, 1, 2, 1, 2), (8, 31, 62, 0, 23, 1, 2, 1, 2), (9, 52, 33, 0, 11, 1, 2, 1, 2), (10, 51, 21, 0, 5, 1, 2, 1, 2), (11, 42, 41, 0, 19, 1, 2, 1, 2), (12, 31, 32, 0, 29, 1, 2, 1, 2), (13, 5, 25, 0, 23, 1, 2, 1, 2), (14, 12, 42, 0, 21, 1, 2, 1, 2), (15, 36, 16, 0, 10, 1, 2, 1, 2), (16, 52, 41, 0, 15, 1, 2, 1, 2), (17, 27, 23, 0, 3, 1, 2, 1, 2), (18, 17, 33, 0, 41, 1, 2, 1, 2), (19, 13, 13, 0, 9, 1, 2, 1, 2), (20, 57, 58, 0, 28, 1, 2, 1, 2), (21, 62, 42, 0, 8, 1, 2, 1, 2), (22, 42, 57, 0, 8, 1, 2, 1, 2), (23, 16, 57, 0, 16, 1, 2, 1, 2), (24, 8, 52, 0, 10, 1, 2, 1, 2), (25, 7, 38, 0, 28, 1, 2, 1, 2), (26, 27, 68, 0, 7, 1, 2, 1, 2), (27, 30, 48, 0, 15, 1, 2, 1, 2), (28, 43, 67, 0, 14, 1, 2, 1, 2), (29, 58, 48, 0, 6, 1, 2, 1, 2), (3

In [23]:
index , x_y , d , q , f , a , li = [] , [] , [] , [] , [] , [] , []
m= L[0][1]#vehicles
customers = L[0][2]
day = L[0][3]
for i in range(3,len(L)-5):
    index.append(int(L[i][0]))
    c = [int(L[i][1]),int(L[i][2])]
    x_y.append(c)
    d.append(int(L[i][3]))
    q.append(int(L[i][4]))
    f.append(int(L[i][5]))
    a.append(int(L[i][6]))
    li.append(tuple(L[i][7:]))

liリストは配達パターン。二進数から１０進数へ変換されている

1日目に配達するなら10で2

2日目に配達するなら01で1

In [24]:
plt.plot(x_y[:, 0], x_y[:, 1], 'o')#"o"は小さい点
plt.plot(x_y[0][0],x_y[0][1],"ro")#デポを描画
plt.show()

TypeError: list indices must be integers or slices, not tuple

In [25]:
import numpy as np
x_y = np.array(x_y)
x = x_y[:, 0]
y = x_y[:, 1]
distance = np.sqrt((x[:, np.newaxis] - x[np.newaxis, :]) ** 2 +(y[:, np.newaxis] - y[np.newaxis, :]) ** 2)

# distance[i, j]が都市iと都市jの距離。10個だけ
print(np.round(distance[:10, :10]))

[[ 0. 14. 21. 33. 17. 14. 11. 26. 22. 23.]
 [14.  0. 12. 19. 31. 22. 17. 23. 12. 24.]
 [21. 12.  0. 15. 37. 21. 28. 35. 22. 16.]
 [33. 19. 15.  0. 50. 36. 35. 35. 21. 31.]
 [17. 31. 37. 50.  0. 20. 21. 37. 38. 33.]
 [14. 22. 21. 36. 20.  0. 25. 40. 33. 12.]
 [11. 17. 28. 35. 21. 25.  0. 16. 18. 34.]
 [26. 23. 35. 35. 37. 40. 16.  0. 14. 46.]
 [22. 12. 22. 21. 38. 33. 18. 14.  0. 36.]
 [23. 24. 16. 31. 33. 12. 34. 46. 36.  0.]]


In [26]:
cost = {}
for i in range(len(x_y)):
    c = 1
    while i+c < len(x_y):
        di = np.sqrt((x_y[i][0] - x_y[i+c][0])**2 + (x_y[i][1] - x_y[i+c][1])**2 )
        cost[i,i+c] = di
        c += 1


# gurobiでとく

In [27]:
model = Model("PVSP")

In [28]:
#変数
x = {}
for i in index:
    for j in index:
        if j > i and i == index[0]:       # depot
            x[i,j] = model.addVar(ub=2, vtype="I", name="x(%s,%s)"%(i,j))
        elif j > i:
            x[i,j] = model.addVar(ub=1, vtype="I", name="x(%s,%s)"%(i,j))
model.update()


In [29]:
#制約
model.addConstr(quicksum(x[index[0],j] for j in index[1:]) == 2*m, "DegreeDepot")
for i in index[1:]:
    model.addConstr(quicksum(x[j,i] for j in index if j < i) +
                    quicksum(x[i,j] for j in index if j > i) == 2, "Degree(%s)"%i)

In [30]:
#目的関数
model.setObjective(quicksum(cost[i,j]*x[i,j] for i in index for j in index if j>i), GRB.MINIMIZE)

In [31]:
#CALLBACK関数
def vrp_callback(model,where):
    if where != GRB.callback.MIPSOL:
        return
    edges = []
    for (i,j) in x:
        if model.cbGetSolution(x[i,j]) > .5:
            if i != V[0] and j != V[0]:
                edges.append( (i,j) )
    G = networkx.Graph()
    G.add_edges_from(edges)
    Components = networkx.connected_components(G)
    for S in Components:
        S_card = len(S)
        q_sum = sum(q[i] for i in S)
        NS = int(math.ceil(float(q_sum)/Q))
        S_edges = [(i,j) for i in S for j in S if i<j and (i,j) in edges]
        if S_card >= 3 and (len(S_edges) >= S_card or NS > 1):
            model.cbLazy(quicksum(x[i,j] for i in S for j in S if j > i) <= S_card-NS)
            print ("adding cut for",S_edges)
    return


In [32]:
model.optimize()

Optimize a model with 52 rows, 1326 columns and 2652 nonzeros
Variable types: 0 continuous, 1326 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 9e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [2e+00, 6e+00]
Found heuristic solution: objective 1678.4825064
Presolve time: 0.01s
Presolved: 52 rows, 1326 columns, 2652 nonzeros
Variable types: 0 continuous, 1326 integer (1275 binary)

Root relaxation: objective 4.219129e+02, 96 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  421.91293    0    6 1678.48251  421.91293  74.9%     -    0s
H    0     0                     434.1649116  421.91293  2.82%     -    0s
H    0     0                     432.1761902  421.91293  2.37%     -    0s
     0     0  422.68567    0    8  432.17619  422.68567  2.20%     -    0s
     0     0  422.68567    

In [33]:
edges = []
for (i,j) in x:
    if x[i,j].X > .5:
        if i != index[0] and j != index[0]:
            edges.append( (i,j) )

print ("Optimal solution:",model.ObjVal)
print ("Edges in the solution:")
print (sorted(edges))

Optimal solution: 423.33959927518794
Edges in the solution:
[(1, 22), (1, 32), (2, 16), (2, 29), (3, 20), (3, 36), (4, 17), (4, 18), (5, 38), (5, 49), (6, 14), (6, 48), (7, 23), (7, 43), (8, 26), (8, 48), (9, 49), (9, 50), (10, 30), (10, 39), (11, 32), (11, 38), (12, 37), (12, 47), (13, 25), (13, 41), (14, 25), (15, 44), (15, 45), (16, 50), (17, 37), (18, 47), (19, 40), (19, 42), (20, 35), (21, 29), (21, 34), (22, 28), (23, 24), (24, 43), (26, 31), (28, 31), (30, 34), (33, 39), (33, 45), (35, 36), (40, 41), (42, 44)]


In [34]:
import math
import random
import networkx
from gurobipy import *

def vrp(V,c,m,q,Q):
    
    def vrp_callback(model,where):
        if where != GRB.callback.MIPSOL:
            return
        edges = []
        for (i,j) in x:
            if model.cbGetSolution(x[i,j]) > .5:
                if i != V[0] and j != V[0]:
                    edges.append( (i,j) )
        G = networkx.Graph()
        G.add_edges_from(edges)
        Components = networkx.connected_components(G)
        for S in Components:
            S_card = len(S)
            q_sum = sum(q[i] for i in S)
            NS = int(math.ceil(float(q_sum)/Q))
            S_edges = [(i,j) for i in S for j in S if i<j and (i,j) in edges]
            if S_card >= 3 and (len(S_edges) >= S_card or NS > 1):
                model.cbLazy(quicksum(x[i,j] for i in S for j in S if j > i) <= S_card-NS)
                #print ("adding cut for",S_edges)
        return


    model = Model("vrp")
    x = {}
    for i in V:
        for j in V:
            if j > i and i == V[0]:       # depot
                x[i,j] = model.addVar(ub=2, vtype="I", name="x(%s,%s)"%(i,j))
            elif j > i:
                x[i,j] = model.addVar(ub=1, vtype="I", name="x(%s,%s)"%(i,j))
    model.update()

    model.addConstr(quicksum(x[V[0],j] for j in V[1:]) == 2*m, "DegreeDepot")
    for i in V[1:]:
        model.addConstr(quicksum(x[j,i] for j in V if j < i) +
                        quicksum(x[i,j] for j in V if j > i) == 2, "Degree(%s)"%i)

    model.setObjective(quicksum(c[i,j]*x[i,j] for i in V for j in V if j>i), GRB.MINIMIZE)

    model.update()
    model.__data = x
    return model,vrp_callback



    
            
if __name__ == "__main__":
    import sys

    V,c,q,Q = index,cost,q,160
    m = 2*m
    model,vrp_callback = vrp(V,c,m,q,Q)

    #model.Params.OutputFlag = 0 # silent mode
    model.params.DualReductions = 0
    model.params.LazyConstraints = 1
    model.optimize(vrp_callback)
    x = model.__data
    
    edges = []
    for (i,j) in x:
        if x[i,j].X > .5:
            if i != V[0] and j != V[0]:
                edges.append( (i,j) )

    print ("Optimal solution:",model.ObjVal)
    print ("Edges in the solution:")
    print (sorted(edges))

Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 52 rows, 1326 columns and 2652 nonzeros
Variable types: 0 continuous, 1326 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 9e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [2e+00, 1e+01]
Presolve time: 0.01s
Presolved: 52 rows, 1326 columns, 2652 nonzeros
Variable types: 0 continuous, 1326 integer (1275 binary)

Root relaxation: objective 4.576452e+02, 73 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  457.64516    0    6          -  457.64516      -     -    0s
     0     0  469.25583    0   24          -  469.25583      -     -    0s
     0     0  470.78539    0   10          -  470.785

In [41]:
import pandas as pd
import random
import time
from gurobipy import Model,quicksum,GRB
import numpy as np
import matplotlib.pyplot as plt
import networkx
import math
import sys
T = [1,2]
def vrp(V,c,m,q,Q,T):
    def vrp_callback(model,where):
        if where != GRB.callback.MIPSOL:#新しい解を発見した時だけ部分巡回路除去制約を追加
            return
        edges = []
        for (i,j,t) in x:
            if model.cbGetSolution(x[i,j,t]) > .5:
                if i != V[0] and j != V[0]:
                    edges.append( (i,j) )
        G = networkx.Graph()
        G.add_edges_from(edges)
        Components = networkx.connected_components(G)
        for S in Components:
            S_card = len(S)
            q_sum = sum(q[i] for i in S)
            NS = int(math.ceil(float(q_sum)/Q))
            S_edges = [(i,j) for i in S for j in S if i<j and (i,j) in edges]
            if S_card >= 3 and (len(S_edges) >= S_card or NS > 1):
                model.cbLazy(quicksum(x[i,j,t] for i in S for j in S if j > i) <= S_card-NS)
                #print ("adding cut for",S_edges)
        return

    

    model = Model("vrp")
    x = {}
    for i in V:
        for j in V:
            for t in T:
                if j > i and i == V[0]:       # depot
                    x[i,j,t] = model.addVar(ub=2, vtype="I", name="x(%s,%s,%s)"%(i,j,t))
                elif j > i:
                    x[i,j,t] = model.addVar(ub=1, vtype="I", name="x(%s,%s.%s)"%(i,j,t))
    model.update()

    for t in T:
        model.addConstr(quicksum(x[V[0],j,t] for j in V[1:]) == 2*m, "DegreeDepot")
    for i in V[1:]:
            model.addConstr(quicksum(x[j,i,t] for j in V if j < i for t in T) +
                            quicksum(x[i,j,t] for j in V if j > i for t in T) == 2, "Degree(%s)"%i)

    model.setObjective(quicksum(c[i,j]*x[i,j,t] for i in V for j in V for t in T if j>i), GRB.MINIMIZE)

    model.update()
    model.__data = x
    return model,vrp_callback
if __name__ == "__main__":

    model,vrp_callback = vrp(V,c,m,q,Q,T)

    model.Params.OutputFlag = 0 # silent mode
    model.params.DualReductions = 0
    model.params.LazyConstraints = 1

    start = time.time()
    model.optimize(vrp_callback)
    x = model.__data

    edges = []
    for (i,j,t) in x:
        if x[i,j,t].X > .5:
            if i != V[0] and j != V[0]:
                edges.append( (i,j,t) )

    print ("Optimal solution:",model.ObjVal)
    elapsed_time = time.time() - start
    gap = model.MIPGap
    #print("==============================")
    #print(w)
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")
    print ("Edges in the solution:")
    print (sorted(edges))



Optimal solution: 459.1871323504884
elapsed_time:0.028918027877807617[sec]
Edges in the solution:
[(1, 22, 1), (1, 22, 2), (2, 16, 1), (2, 29, 1), (3, 20, 1), (3, 20, 2), (4, 18, 1), (4, 18, 2), (5, 38, 1), (5, 38, 2), (6, 48, 1), (7, 23, 1), (7, 23, 2), (8, 26, 1), (8, 26, 2), (9, 49, 1), (9, 49, 2), (10, 39, 1), (10, 39, 2), (13, 41, 1), (13, 41, 2), (14, 25, 1), (14, 25, 2), (15, 44, 1), (15, 44, 2), (16, 50, 1), (17, 37, 1), (17, 37, 2), (19, 40, 2), (19, 42, 1), (21, 29, 1), (21, 50, 1), (24, 43, 1), (24, 43, 2), (28, 31, 1), (28, 31, 2), (30, 34, 1), (30, 34, 2), (33, 45, 1), (33, 45, 2), (35, 36, 1), (35, 36, 2), (40, 42, 1)]
